In [1]:
%reset

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import pandas as pd
import pickle
import copy
import pandas as pd
from tqdm.notebook import tqdm_notebook
import os

In [3]:
mat_dir = 'D:\\AnatArziData\\OExpOut\\eventDetection\\try2\\imported_eventDetectionChan\\no_filters\\imported_scoringPrepro_mats'
out_dir = 'D:\\AnatArziData\\OExpOut\\eventDetection\\try2\\imported_eventDetectionChan\\no_filters\\pkls'
prepro_name = "imported"
import_type = "eventDetectionChan"
dir_scoring = "C:\\Users\\User\\OneDrive - huji.ac.il\\AnatArzData\\Data\\FinalScores"

In [4]:
all_subjects = np.array(['01','02','03','04','05','06','07','08', '09', '10','11', '12','13', '14', '15', '16','17','18', '19', '20', '21','22','23','24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34','35', '36','37','38'])

outlier_subs_major_exp_fault = ['18']
valid_subs = np.delete(all_subjects,np.isin(all_subjects,outlier_subs_major_exp_fault))

cfg = {}
cfg['outputs_dir_path'] =  'C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput'
cfg['mat_files_dir'] =  mat_dir

cfg['prepro_name'] = prepro_name
cfg['subs'] = valid_subs
cfg['outlier_subs'] = outlier_subs_major_exp_fault

## Save - all subjects, in 10 subs butches

In [28]:
# save in parts
parts = [[0,10],[11,20],[20,30],[30,valid_subs.size]]

for i_part, part in enumerate(parts):
    subs = valid_subs[part[0]:part[1]]
    all_files_dic = {}
    for sub in subs:
        for sleep_i in np.arange(1,7):
            sub_load_filepath = f"{cfg['mat_files_dir']}\\sub{sub}_sleep{sleep_i}_{prepro_name}_{import_type}.mat"
            try: 
                matlabImport = scipy.io.loadmat(sub_load_filepath, simplify_cells=True)
                all_files_dic[f'{sub}_{sleep_i}'] = {}
                all_files_dic[f'{sub}_{sleep_i}']['events'] = pd.DataFrame(matlabImport['subEvents'])
                all_files_dic[f'{sub}_{sleep_i}']['data'] = matlabImport[f'subData']
                all_files_dic[f'{sub}_{sleep_i}']['subnum'] = sub
                all_files_dic[f'{sub}_{sleep_i}']['filenum'] = sleep_i

                datafile_scoring = f"{dir_scoring}\\s_{sub}_{sleep_i}"
                readEx = pd.read_excel(f'{datafile_scoring}.xlsx',header=None) # "header =None" is important, otherwise the scoring will be shifted
                all_files_dic[f'{sub}_{sleep_i}']['scoring'] = [x[0] for x in readEx.to_numpy()]

                cfg['sample_freq'] = matlabImport[f'samplefreq']
                cfg['electrodes_names'] = matlabImport[f'chansNames']

            except Exception: continue #print(f'Error importing sub file at: {sub_load_filepath}')
    with open(f'{out_dir}\\{import_type}_{i_part}.pkl', 'wb') as file:
        pickle.dump([all_files_dic, cfg], file)

In [23]:
# with open(f'{mat_dir}\\{import_type}_1-10.pkl', 'wb') as file:
#     pickle.dump([all_files_dic, cfg], file)

# Load
# with open(f'{epochs_dir}\\{import_type}_allEvents_df.pkl', 'rb') as file:
#     _allEvents_df = pickle.load(file)
# with open(f'{epochs_dir}\\{import_type}_allEpochs.pkl', 'rb') as file:
#     _allEpochs_perCond = pickle.load(file)

## Save - file per subject

In [6]:
subs = all_subjects #np.array(['30','31','32','33','34','35','36','37','38'])
subs =  np.array(['13'])

for sub in subs:
    for sleep_i in np.arange(1,7):
        file_dic = {}
        sub_load_filepath = f"{cfg['mat_files_dir']}\\sub{sub}_sleep{sleep_i}_{prepro_name}_{import_type}.mat"
        try: 
            matlabImport = scipy.io.loadmat(sub_load_filepath, simplify_cells=True)
        except Exception: continue #print(f'Error importing sub file at: {sub_load_filepath}')
        file_dic[f'{sub}_{sleep_i}'] = {}
        file_dic[f'{sub}_{sleep_i}']['events'] = pd.DataFrame(matlabImport['subEvents'])
        file_dic[f'{sub}_{sleep_i}']['data'] = matlabImport[f'subData']
        file_dic[f'{sub}_{sleep_i}']['subnum'] = sub
        file_dic[f'{sub}_{sleep_i}']['filenum'] = sleep_i

        datafile_scoring = f"{dir_scoring}\\s_{sub}_{sleep_i}"
        readEx = pd.read_excel(f'{datafile_scoring}.xlsx',header=None) # "header =None" is important, otherwise the scoring will be shifted
        file_dic[f'{sub}_{sleep_i}']['scoring'] = [x[0] for x in readEx.to_numpy()]

        cfg['sample_freq'] = matlabImport[f'samplefreq']
        cfg['electrodes_names'] = matlabImport[f'chansNames']
        
        with open(f'{out_dir}\\{import_type}_sub-{sub}_sleep-{sleep_i}.pkl', 'wb') as file:
            pickle.dump([file_dic, cfg], file)